# Virtual species

In this vignette, we provide a demonstration of how the different **SpeciesDistributionToolkit** functions can be chained together to rapidly create a virtual species, generate its range map, and sample points from it according to the predicted suitability.

In [12]:
using SpeciesDistributionToolkit
using CairoMakie
using Statistics
CairoMakie.activate!(px_per_unit = 6.0)

We start by defining the extent in which we want to create the virtual species. For the purpose of this example, we will use the country of Austria, a polygon of which is available in the GADM database. Note that the `boundingbox` function returns the coordinates *in WGS84*.

In [ ]:
place = SpeciesDistributionToolkit.gadm("PRY")
extent = SpeciesDistributionToolkit.boundingbox(place)

We then download some environmental data. In this example, we use the BioClim variables as distributed by CHELSA. In order to simplify the code, we will only use BIO1 (mean annual temperature) and BIO12 (total annual precipitation). Note that we collect these layers in a vector typed as `SDMLayer{Float32}`, in order to ensure that future operations already recevie floating point values.

In [ ]:
provider = RasterData(CHELSA2, BioClim)
L = SDMLayer{Float32}[SDMLayer(provider; layer=l, extent...) for l in ["BIO1", "BIO12"]]

We now mask the layers using the polygons we downloaded initially. Here, this is done in two steps, first the masking of the first layer, and second the masking of all other layers. Currently unreleased versions of the package have a shortcut for this operation.

In [ ]:
msk = mask!(L[1], place)
rescale!.([mask!(l, msk) for l in L])

In the next steps, we will generate some virtual species. These are defined by an environmental response to each layer, linking the value of the layer at a point to the suitability score. For the sake of expediency, we only use logistic responses, and generate one function for each layer (drawing $\alpha$ from a normal distribution, and $\beta$ uniformly).

In [ ]:
logistic(x, α, β) = 1 / (1 + exp((x-β)/α))
logistic(α, β) = (x) -> logistic(x, α, β)
f = [logistic(randn(), rand()) for _ in eachindex(L)]

In the next step, we create a layer of suitability, by applying the logistic function to each environmental variable layer, and taking the product of all suitabilities:

In [ ]:
S = prod([f[i].(L[i]) for i in eachindex(L)])

In order to generate the range of the species, we set a target prevalence, and identify the quantile corresponding to this prevalence in the suitability layer.

In [ ]:
target_prevalence = 0.3268
cutoff = quantile(S, 1-target_prevalence)

Random observations for the virtual species are generated by setting the probability of inclusion to 0 for all values above the cutoff, and then sampling proportionally to the suitability for all remaining points. Note that the method is called `backgroundpoints`, as it is normally used for pseudo-absences. The second argument of this method is the number of points to generate.

In [ ]:
presencelayer = backgroundpoints((v -> v > cutoff ? v : 0.0).(S), 59)

We can finally plot the result:

In [ ]:
f = Figure(size=(700, 700))
ax = Axis(f[1,1], aspect=DataAspect())
heatmap!(ax, S .> cutoff, colormap=["#cececebb", :green])
lines!(ax, place[1].geometry, color=:black)
scatter!(ax, presencelayer, color=:white, strokecolor=:black, strokewidth=2, markersize=10, label="Virtual presences")
tightlimits!(ax)
hidespines!(ax)
hidedecorations!(ax)
axislegend(ax, position=:lb, framevisible=false)
f